In [1]:
#do not run unless tensorflow can't find cuda
import os
os.environ['CUDA_DIR'] = "/usr/lib/cuda"
os.environ['XLA_FLAGS'] = "--xla_gpu_cuda_data_dir=/usr/lib/cuda"

In [2]:
import tensorflow as tf

2024-05-05 00:09:27.615466: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-05 00:09:27.638006: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 00:09:27.638023: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 00:09:27.638915: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-05 00:09:27.644172: I tensorflow/core/platform/cpu_feature_guar

# Define and Load the VAE-GANs for healthy and fractured vertebra.

In [3]:
class CVAE(tf.keras.Model):
  """Convolutional variational autoencoder."""

  def __init__(self, latent_dim):
    super(CVAE, self).__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(128, 128, 1)),
            tf.keras.layers.Conv2D(
                filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
            # No activation
            tf.keras.layers.Dense(latent_dim + latent_dim),
        ]
    )

    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(units=16*16*32, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(16, 16, 32)),
            tf.keras.layers.Conv2DTranspose(
                filters=128, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=32, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            # No activation
            tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=3, strides=1, padding='same'),
        ]
    )

  @tf.function
  def sample(self, eps=None):
    if eps is None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)

  def encode(self, x):
    mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  def decode(self, z, apply_sigmoid=False):
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits

  def generate(self, mean, logvar):
    z = self.reparameterize(mean, logvar)
    x_logit = self.decode(z, apply_sigmoid=True)
    return x_logit

In [4]:
latent_dim = 100
generator_healthy = CVAE(latent_dim)
generator_healthy.load_weights('./real_healthy_cts_transfer_variational_autoencoder_generator')

latent_dim = 100
generator_fractured = CVAE(latent_dim)
generator_fractured.load_weights('./real_fractured_cts_transfer_variational_autoencoder_generator')

2024-05-05 00:09:28.552092: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-05 00:09:28.580987: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-05 00:09:28.581131: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

1. Load the images and their categories into memory. Save the images in the images[] array and the categories in the categories[] array

In [5]:
import os
import os.path as osp
import cv2
import numpy as np
import pandas as pd
path = '../cropped_healthy_vertebra'

categories = []
images = []

for image_file in os.listdir(path):
            img = cv2.imread(osp.join(path, image_file))
            hh, ww = img.shape[:2]
            maximum = max(hh, ww)
            img = cv2.resize(img,(128,128))

            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            images.append(img)
            categories.append('healthy')
        
path = '../cropped_fractured_vertebra'
for image_file in os.listdir(path):
            img = cv2.imread(osp.join(path, image_file))
            hh, ww = img.shape[:2]
            maximum = max(hh, ww)
            
            img = cv2.resize(img,(128,128))

            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            images.append(img)
            categories.append('fractured')        

images = np.array(images)
categories = np.array(categories)

In [6]:
images = images/255

2. Convert the images into numpy arrays, and perform one hot encoding on the categories

In [7]:
X = images
y = categories

4. Split the data between training and testing

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(X, y , test_size = 0.2,stratify=y, random_state=0)

In [9]:
X_train = list(X_train)
y_train = list(y_train)

5) Generate N healthy CTs and N fractured CTs. For the case of no augmentation, N is set to 0.

In [10]:
N = 25000
from tqdm import tqdm
for i in tqdm(range(N)):
            noise_mean = tf.random.normal([1, latent_dim])
            noise_logvar = tf.random.normal([1, latent_dim])
            
            generated_image = generator_healthy.generate(noise_mean, noise_logvar)
            X_train.append(np.reshape(generated_image, (128,128)))
            y_train.append('healthy')
            
for i in tqdm(range(N)):
            noise_mean = tf.random.normal([1, latent_dim])
            noise_logvar = tf.random.normal([1, latent_dim])
            generated_image = generator_fractured.generate(noise_mean, noise_logvar)
            X_train.append(np.reshape(generated_image, (128,128)))
            y_train.append('fractured') 

100%|██████████| 25000/25000 [01:05<00:00, 379.92it/s]


In [11]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [12]:
# X_train = (X_train) / 255
# X_test = (X_test)/255

In [13]:
y_train = pd.get_dummies(pd.DataFrame(y_train))
y_test = pd.get_dummies(pd.DataFrame(y_test))

In [14]:
X_train.shape

(50523, 128, 128)

6. Build an initial neural network and print its architecture

In [15]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dropout, Dense
from tensorflow.keras import Sequential

model = Sequential()
model.add(Conv2D(32, 3, activation='relu', input_shape=(128,128,1)))
model.add(MaxPool2D())
model.add(Dropout(0.3))
model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 30, 30, 64)       

7. Compile and train the initial neural network

In [16]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))


Epoch 1/20


2024-05-05 00:11:53.344417: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_4/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-05-05 00:11:53.926698: I external/local_xla/xla/service/service.cc:168] XLA service 0x7197a8838240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-05 00:11:53.926715: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-05-05 00:11:53.929786: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1714857113.980865  939010 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1579/1579 [==============================] - 7s 3ms/step - loss: 0.3063 - accuracy: 0.8634 - val_loss: 0.3768 - val_accuracy: 0.8626
Epoch 2/20
1579/1579 [==============================] - 5s 3ms/step - loss: 0.1306 - accuracy: 0.9493 - val_loss: 0.4514 - val_accuracy: 0.8702
Epoch 3/20
1579/1579 [==============================] - 5s 3ms/step - loss: 0.0822 - accuracy: 0.9684 - val_loss: 0.3873 - val_accuracy: 0.8779
Epoch 4/20
1579/1579 [==============================] - 5s 3ms/step - loss: 0.0636 - accuracy: 0.9764 - val_loss: 0.5041 - val_accuracy: 0.8855
Epoch 5/20
1579/1579 [==============================] - 5s 3ms/step - loss: 0.0507 - accuracy: 0.9815 - val_loss: 0.5162 - val_accuracy: 0.8855
Epoch 6/20
1579/1579 [==============================] - 5s 3ms/step - loss: 0.0431 - accuracy: 0.9841 - val_loss: 0.5701 - val_accuracy: 0.9008
Epoch 7/20
1579/1579 [==============================] - 5s 3ms/step - loss: 0.0361 - accuracy: 0.9863 - val_loss: 0.6521 - val_accuracy: 0.8855
Epo

8. Calculate the classification metrics.

In [18]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

y_true = np.argmax(np.array(y_test), axis=1)

y_predicted = np.argmax(model.predict(X_test),axis=1)

target_names = ['Healthy','Fractured']

print(classification_report(y_true, y_predicted, target_names=target_names))
print('The accuracy is',accuracy_score(y_true, y_predicted))

5/5 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

     Healthy       0.85      0.89      0.87        56
   Fractured       0.92      0.88      0.90        75

    accuracy                           0.89       131
   macro avg       0.88      0.89      0.88       131
weighted avg       0.89      0.89      0.89       131

The accuracy is 0.8854961832061069


In [30]:
y_test

,0_benign,0_malignant
2,1,0
396,0,1
440,0,1
142,1,0
29,1,0
...,...,...
379,0,1
458,0,1
90,1,0
325,0,1
